In [ ]:
from helper import *
import subprocess
import os
import sys
import time
import csv
import math
import numpy as np
import pandas as pd

In [ ]:
# simply put osm files into the data-dir (all will get evaluated)
EVAL_DIR = "/home/felix/todo/osm-tmp"
SRTM_DIR = "/home/felix/todo/osm/srtm"
MLP_METHODS = ["kmeans", "merge", "hop"]
# MLP_LAYERS = [[int(2 ** i)] for i in np.arange(8.0, 13.5, 0.5)]
MLP_LAYERS = [[int(2 ** i)] for i in np.arange(8.0, 13.5, 1.0)]
QUERY_METHODS = ["normal", "bi", "crp", "pch", "prp"]
EVAL_FILE = os.path.join(EVAL_DIR, "saarland.eval")
EVAL_AMOUNTS = 10_000
print(MLP_METHODS, "with", MLP_LAYERS)

# generate

In [ ]:
# create logfile
if not os.path.isfile(EVAL_DIR + "/log.json"):
    os.create(EVAL_DIR + "/log.json")
# make sure everything compile
shell_execute(["cargo", "build", "--release"], EVAL_DIR)

## extract

In [ ]:
# REMINDER: make sure the correct settings in "pbfextractor" are correct
OSM_FILES = find_files_ending(".osm.pbf", EVAL_DIR)
FMI_FILES = out_files(OSM_FILES, "-latest.osm.pbf", ".fmi")
print("evaluate on", len(OSM_FILES), "OSM-file(s)")
for osm_file, fmi_file in zip(OSM_FILES, FMI_FILES):
    if not_created_yet(fmi_file, EVAL_DIR):
        fmi_longname = fmi_file.replace(".fmi", "-full.txt")
        shell_execute(["cargo", "run", "--release", "--bin", "pbfextractor", "--", osm_file, SRTM_DIR, fmi_longname], EVAL_DIR)
        shell_execute(["cargo", "run", "--release", "--bin", "fmi_largest_set", "--", "-i", fmi_longname, "-o", fmi_file], EVAL_DIR)

## eval-file

In [ ]:
FMI_FILES = find_files_ending(".fmi", EVAL_DIR)
EVAL_FILES = out_files(FMI_FILES, ".fmi", ".eval")
for (fmi_file, eval_file) in zip(FMI_FILES, EVAL_FILES):
    if not_created_yet(eval_file, EVAL_DIR):
        shell_execute([sys.executable, "../analysis/generate-eval-file.py", "-i", fmi_file, "-c", str(EVAL_AMOUNTS), "-o", eval_file], EVAL_DIR)

## MLP

In [ ]:
PRE_FILES = list()
for fmi_file in find_files_ending(".fmi", EVAL_DIR):
    for method in MLP_METHODS:
        for level in MLP_LAYERS:
            PRE_FILES.append([fmi_file, fmi_file.replace(".fmi", "") + "-",method , "-", "_".join(map(str, level)), ".mlp"])
for pre_file_list in PRE_FILES:
    fmi_file = pre_file_list[0]
    mlp_file = "".join(pre_file_list[1:])
    method = pre_file_list[2]
    level = pre_file_list[4]
    if not_created_yet(mlp_file, EVAL_DIR):
        levels = level.split("_")
        command = ["cargo", "run", "--release", "--bin", "mlp_" + method, "--", "-f", fmi_file, "-o", mlp_file, "-p"]
        for level_index in range(len(levels)):
            if method == "merge":
                value = math.prod(map(int, levels[:(level_index + 1)]))
            else:
                value = levels[level_index]
            command.append(str(value))
        shell_execute(command, EVAL_DIR)

# pre

In [ ]:
combinations = dict()
for fmi_file in find_files_ending(".fmi", EVAL_DIR):
    for method in MLP_METHODS:
        for level in MLP_LAYERS:
            combinations[(fmi_file, method, "_".join(map(str, level)))] = {"fmi_file": fmi_file, "mlp_file": fmi_file.replace(".fmi", "") + "-" + method + "-" + "_".join(map(str, level)) + ".mlp"}

## mlp-csv-export

In [ ]:
for ((fmi_file, method, partitions), inputs) in combinations.items():
    output = inputs["mlp_file"].replace(".mlp", ".csv")
    if not_created_yet(output, EVAL_DIR):
        shell_execute([sys.executable, "../analysis/fmi+mlp_csv-export.py", "-f", inputs["fmi_file"], "-m", inputs["mlp_file"], "-o", output], EVAL_DIR)

## precalculation

In [ ]:
FMI_FILES = find_files_ending(".fmi", EVAL_DIR)
MLP_FILES = find_files_ending(".mlp", EVAL_DIR)

for mlp_file in MLP_FILES:
    # get fmi-filename and check if it exists
    fmi_file = "-".join(mlp_file.split("-")[:-2]) + ".fmi"
    if fmi_file in FMI_FILES:
        bin_file = mlp_file.replace(".mlp", ".bin")
        if not_created_yet(bin_file, EVAL_DIR):
            shell_execute(["cargo", "run", "--bin", "prp_pre", "--release", "--", "-f", fmi_file, "-m", mlp_file, "-o", bin_file], EVAL_DIR)

## pch-precalculation

In [ ]:
FMI_FILES = find_files_ending(".fmi", EVAL_DIR)
MLP_FILES = find_files_ending(".mlp", EVAL_DIR)

for mlp_file in find_files_ending(".mlp", EVAL_DIR):
    fmi_file = "-".join(mlp_file.split("-")[:-2]) + ".fmi"
    if fmi_file in FMI_FILES:
        with open(fmi_file) as csvfile:
            fmireader = csv.reader(csvfile, delimiter=' ')
            tmp = next(fmireader)
            while len(tmp) == 0 or tmp[0].startswith("#"):
                tmp = next(fmireader)
            amount_nodes_fmi = int(next(fmireader)[0])
        csv_file = mlp_file.replace(".mlp", ".csv")
        df = pd.read_csv(csv_file)
        contracted_nodes = sum(df.highest_diff.value_counts().loc[1:])
        bin_file = mlp_file.replace(".mlp", "-pch.bin")
        percentage = 1 - (contracted_nodes / amount_nodes_fmi)
        if not_created_yet(bin_file, EVAL_DIR):
            shell_execute(["cargo", "run", "--bin", "prp_pre", "--release", "--", "-f", fmi_file, "-p", str(percentage), "-o", bin_file], EVAL_DIR)
    else:
        raise SystemExit("no compatible fmi-file found!")

## evaluate

In [ ]:
BIN_FILES = find_files_ending(".bin", EVAL_DIR)
UNIQUE_BIN_FILES = set(["-".join(f.split("-")[:-2]) for f in BIN_FILES])
CORE_BIN_FILES = list()
for unique_bin_file in UNIQUE_BIN_FILES:
    for bin_file in BIN_FILES:
        if bin_file.startswith(unique_bin_file):
            CORE_BIN_FILES.append(bin_file)
            break
for method in QUERY_METHODS:
    if method == "normal" or method == "bi":
        for file in CORE_BIN_FILES:
            out_file = file
            for mlp_method in MLP_METHODS:
                tmp = "-" + mlp_method
                if out_file.find(tmp) != -1:
                    out_file = out_file[:out_file.index(tmp)]
            time_file = out_file + "-"+ method + "-time.json"
            if not_created_yet(time_file, EVAL_DIR):
                shell_execute(["cargo", "run", "--bin", "prp_eval", "--release", "--", "-f", file, "-e", EVAL_FILE, "-x", time_file, "-m", method, "-t", "time"], EVAL_DIR)
            count_file = out_file + "-"+ method + "-count.json"
            if not_created_yet(count_file, EVAL_DIR):
                shell_execute(["cargo", "run", "--bin", "prp_eval", "--release", "--", "-f", file, "-e", EVAL_FILE, "-x", count_file, "-m", method, "-t", "count"], EVAL_DIR)
    else:
        for file in BIN_FILES:
            if file.endswith("-pch.bin"):
                if method == "pch":
                    time_file = file.replace(".bin", "-"+ method + "-time.json")
                    if not_created_yet(time_file, EVAL_DIR):
                        shell_execute(["cargo", "run", "--bin", "prp_eval", "--release", "--", "-f", file, "-e", EVAL_FILE, "-x", time_file, "-m", method, "-t", "time"], EVAL_DIR)
            else:
                time_file = file.replace(".bin", "-"+ method + "-time.json")
                if not_created_yet(time_file, EVAL_DIR):
                    shell_execute(["cargo", "run", "--bin", "prp_eval", "--release", "--", "-f", file, "-e", EVAL_FILE, "-x", time_file, "-m", method, "-t", "time"], EVAL_DIR)
                count_file = file.replace(".bin", "-"+ method + "-count.json")
                if not_created_yet(count_file, EVAL_DIR):
                    shell_execute(["cargo", "run", "--bin", "prp_eval", "--release", "--", "-f", file, "-e", EVAL_FILE, "-x", count_file, "-m", method, "-t", "count"], EVAL_DIR)